In [1]:
import io, os, sys, types


In [2]:
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [3]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
            

In [4]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [5]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [6]:
sys.meta_path.append(NotebookFinder())

In [8]:
ls nbpackage

 Volume in drive C has no label.
 Volume Serial Number is F450-D4C4

 Directory of C:\Users\custom\Paper2



File Not Found


In [9]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

from IPython.display import display, HTML

formatter = HtmlFormatter()
lexer = PythonLexer()

# publish the CSS for pygments highlighting
display(HTML("""
<style type='text/css'>
%s
</style>
""" % formatter.get_style_defs()
))

In [10]:
def show_notebook(fname):
    """display a short summary of the cells of a notebook"""
    with io.open(fname, 'r', encoding='utf-8') as f:
        nb = read(f, 4)
    html = []
    for cell in nb.cells:
        html.append("<h4>%s cell</h4>" % cell.cell_type)
        if cell.cell_type == 'code':
            html.append(highlight(cell.source, lexer, formatter))
        else:
            html.append("<pre>%s</pre>" % cell.source)
    display(HTML('\n'.join(html)))

show_notebook(os.path.join("nbpackage", "mynotebook.ipynb"))

FileNotFoundError: [Errno 2] No such file or directory: 'nbpackage\\mynotebook.ipynb'

In [11]:
from nbpackage import mynotebook

ModuleNotFoundError: No module named 'nbpackage'

In [12]:
ls


 Volume in drive C has no label.
 Volume Serial Number is F450-D4C4

 Directory of C:\Users\custom\Paper2

10/08/2018  01:07 PM    <DIR>          .
10/08/2018  01:07 PM    <DIR>          ..
10/08/2018  01:01 PM    <DIR>          .ipynb_checkpoints
10/07/2018  04:18 PM             9,234 code1.ipynb
10/08/2018  01:01 PM            15,779 code3.ipynb
10/07/2018  06:06 PM            22,720 Glass_data.xlsx
10/07/2018  06:17 PM             2,548 method_of_moments.ipynb
10/08/2018  01:07 PM            15,265 notebook_as_module.ipynb
               5 File(s)         65,546 bytes
               3 Dir(s)  139,959,406,592 bytes free


In [13]:
import code1.py

0.22132295573711533
1.50611766843
[ 0.5  0.7  0.8]
[[1 2]
 [3 4]]
[[3.5, 10.5], [4, 5]]
[[  3.37205303]
 [ 10.16930347]]
3.37205303038
10.1693034696
[[-0.00032131211361252632], [3.1947626097872384e-05]]
[[3.5], [10.5]]
[[-0.00668672 -0.0192037 ]
 [ 0.0019094   0.00668077]]
Wonderful


ModuleNotFoundError: No module named 'code1.py'; 'code1' is not a package

In [14]:
ls

 Volume in drive C has no label.
 Volume Serial Number is F450-D4C4

 Directory of C:\Users\custom\Paper2

10/08/2018  01:11 PM    <DIR>          .
10/08/2018  01:11 PM    <DIR>          ..
10/08/2018  01:01 PM    <DIR>          .ipynb_checkpoints
10/08/2018  01:10 PM    <DIR>          __pycache__
10/07/2018  04:18 PM             9,234 code1.ipynb
10/08/2018  01:10 PM             1,791 code1.py
10/08/2018  01:01 PM            15,779 code3.ipynb
10/07/2018  06:06 PM            22,720 Glass_data.xlsx
10/07/2018  06:17 PM             2,548 method_of_moments.ipynb
10/08/2018  01:10 PM               573 method_of_moments.py
10/08/2018  01:11 PM            17,739 notebook_as_module.ipynb
               7 File(s)         70,384 bytes
               4 Dir(s)  139,959,222,272 bytes free


In [16]:
from code1 import G

In [17]:
G(4,5)

[[0.16631094104308383, -0.11751201469403141],
 [-0.11751201469403141, 0.10381094104308392]]